In [1]:
%load_ext autoreload
%autoreload 2

from spot.utils import *
os.chdir(proj_root())

In [2]:
from spot.function_dataset import data_project_from_dir
from spot.static_analysis import SignatureErrorAnalysis, AccuracyMetric
from spot.model import ModelWrapper
from spot.experiments.typet5 import TypeT5Configs

# dataset_name = "InferTypes4Py"
dataset_name = "ManyTypes4Py"

# load projects
split = "test"
repos_dir = get_dataset_dir(dataset_name) / "repos" / split
repo_paths = [f for f in repos_dir.iterdir() if f.is_dir()]
projects = pmap(
    data_project_from_dir,
    repo_paths,
    desc="Loading projects",
)

labels = {p.name: p.get_sigmap() for p in projects}
model_path = get_model_dir() / TypeT5Configs.Default.get_model_name()
# common_names can be obtained in other ways, but here we just load it from the model dir
common_names = ModelWrapper.load_common_type_names(model_path)
metric = AccuracyMetric(common_names, relaxed_equality=False, filter_none_any=False, name="full_acc")

print("Type slots:", sum(sig.n_annots() for p in projects for sig in p.get_sigmap().values()))
pretty_print_dict(SignatureErrorAnalysis(labels, labels, metric).accuracies)

/home/jiayi/Projects/SPOT/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Loading projects: 100%|██████████| 50/50 [00:25<00:00,  1.99it/s]


Type slots: 28436
full_acc: 100.00% (count=15.8k)
full_acc_by_cat:
   FuncArg: 100.00% (count=8.0k)
   FuncReturn: 100.00% (count=5.8k)
   ClassAtribute: 100.00% (count=1.9k)
   GlobalVar: 100.00% (count=107)
full_acc_by_simple:
   complex: 100.00% (count=3.3k)
   simple: 100.00% (count=12.4k)
full_acc_label_size: 1.4205
full_acc_pred_size: 1.4205
full_acc_ignored_labels: 0


In [ ]:
# parse all type annotations
def get_repo_annots(repo: GitRepo):
    annots = []
    file_to_annots = repo.collect_annotations(repos_dir, silent=True)
    for d in file_to_annots.values():
        annots.extend(d.values())
    return annots

with ProcessPoolExecutor(max_workers=20) as executor:
    all_annots = list(seq_flatten(parallel_map_unordered(get_repo_annots, useful_repos, executor)))

n_total = len(all_annots)
all_annots = [t for t in all_annots if t is not None]
n_failed_to_parse = n_total - len(all_annots)
print("total number of parsed annotations: ", len(all_annots))
print(f"{n_failed_to_parse / n_total * 100:.3f}% failed to parse.")

100%|██████████| 663/663 [01:04<00:00, 10.29it/s]

total number of parsed annotations:  343537
0.004657214461815207% failed to parse.


In [ ]:
from collections import Counter

def freq_table(labels, at_least=0):
    counts = Counter(labels).most_common(None)
    rows = filter(lambda row: row[1] >= at_least, counts)
    return pd.DataFrame(rows, columns=["label", "count"])

def plot_counts(labels, title:str="", top_k=100):
    freq = freq_table(labels)
    subset = freq[0:top_k]
    coverage = sum(subset["count"]) / sum(freq["count"])
    print(f"Number of different labels: {len(freq)}")
    print(f"Top-{top_k} labels achieve coverage: {coverage * 100:.2f}%")
    data = pd.DataFrame(subset)
    fig = px.bar(data, x="label", y="count", title=title)
    fig.update_xaxes(visible=True, showticklabels=False)
    return fig

def type_name(xs):
    if xs == ():
        return "[empty]"
    else:
        return xs[-1]

In [ ]:
plot_counts((map(str,all_annots)), title="Full Types")

Number of different labels: 26864
Top-100 labels achieve coverage: 64.00%


In [ ]:
plot_counts((map(lambda a: type_name(a.head), all_annots)), title="Partial Types")

Number of different labels: 9668
Top-100 labels achieve coverage: 80.67%


In [ ]:
all_heads = [type_name(h) for t in all_annots for h in t.all_heads()]
plot_counts(all_heads, title="Type Constructor Names")

Number of different labels: 11894
Top-100 labels achieve coverage: 79.30%


In [ ]:
freq_table(all_heads, at_least=3)

,value,count
0,str,92221
1,None,51495
2,int,31179
3,Optional,26784
4,Any,20776
...,...,...
5982,FragmentPagerInfo,3
5983,Measure,3
5984,LinesUpdate,3
5985,LineDisplay,3
